In [4]:
# 使用sys.path添加上级目录
import sys
import os
package_path = os.path.dirname(os.path.dirname(os.getcwd()))
file_path = os.path.join(package_path, "ch07", "01_main-chapter-code")
print(file_path)
sys.path.append(file_path)

import torch
if torch.cuda.is_available():
   device = torch.device("cuda")
elif torch.backends.mps.is_available():
   device = torch.device("mps")
else:
   device = torch.device("cpu")

F:\project\LLMs-from-scratch-CN\ch07\01_main-chapter-code


In [5]:
from importlib.metadata import version

pkgs = ["tqdm",    # 进度条
        ]

for p in pkgs:
    print(f"{p} version: {version(p)}")

tqdm version: 4.67.1


# 【todo】使用Ollama的REST API

# 使用deepseek构造数据集

In [6]:
from openai import OpenAI
from config_manager import config_manager

config = config_manager.config
api_key = config.get('api', {}).get('deepseek_api_key')
if not api_key:
    raise ValueError("需要提供API密钥")

# 初始化OpenAI客户端来访问Deepseek API
client = OpenAI(
    api_key=api_key,
    base_url=config.get('api', {}).get('deepseek_api_url')
)

In [7]:
import urllib.request
import json

def query_model(prompt, client, config, batch_mode=False):
    messages = [
        {
            "role": "user",
            "content": prompt
        }
    ]
    if batch_mode:
        messages = []
        for pt in prompt:
            messages.append({
                "role": "user",
                "content": prompt
            })
    # 构建请求数据
    data = {
        "model": config.get('api', {}).get('model', 'deepseek-chat'),
        "messages": messages,
        "temperature": 0.
    }
    # 发送请求
    response = client.chat.completions.create(**data)
    # 解析响应
    try:
        content = response.choices[0].message.content
        # # 清理可能的前导和尾随空白字符
        # content = content.strip()
        
        # # 移除可能的Markdown代码块标记
        # if content.startswith('```json'):
        #     content = content[7:]  # 移除开头的```json
        # if content.endswith('```'):
        #     content = content[:-3]  # 移除结尾的```
        
        # 清理并解析JSON
        content = content.strip()
        response_data = json.loads(content)

        return response_data
        
    except json.JSONDecodeError as e:
        # print(f"JSON解析失败: {content}")
        # print(f"错误信息: {str(e)}")
        return content

result = query_model("What do Llamas eat?", client, config)
print(result)

Llamas are herbivores with a diet primarily consisting of grasses, hay, and other plant materials. Here’s a breakdown of what they typically eat:

### **1. Main Diet:**
- **Grass & Hay** – The bulk of their diet, especially in captivity. They graze on fresh pasture or are fed high-quality grass hay (e.g., timothy, orchard, or brome hay).
- **Forage** – They browse on leaves, shrubs, and other available vegetation in their natural habitat (Andes Mountains).

### **2. Supplemental Foods:**
- **Pellets/Grain Mixes** – Specially formulated llama feed (in moderation) can provide extra nutrients.
- **Vegetables & Fruits** – Occasionally, they can have small amounts of carrots, apples, or leafy greens as treats (avoid sugary or high-oxalate foods like spinach).

### **3. Important Notes:**
- **Fresh Water** – Must always be available.
- **Avoid Overfeeding Grains** – Too much grain can cause digestive issues like bloat.
- **Mineral Licks** – Llamas may need salt or mineral supplements, especi

# 加载JSON数据

In [8]:
from pathlib import Path

json_file = Path("..", "..", "ch07", "01_main-chapter-code", "instruction-data.json")

with open(json_file, "r") as file:
    json_data = json.load(file)

print("Number of entries:", len(json_data))

Number of entries: 1100


In [9]:
json_data[0]

{'instruction': 'Evaluate the following phrase by transforming it into the spelling given.',
 'input': 'freind --> friend',
 'output': 'The spelling of the given phrase "freind" is incorrect, the correct spelling is "friend".'}

In [10]:
# 格式化输入
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. Write a response that "
        f"appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
    instruction_text + input_text

    return instruction_text + input_text

In [11]:
# 测试生成数据
import random

for entry in json_data[:5]:
    politeness = random.choice(["polite", "impolite"])
    prompt = (
        f"Given the input `{format_input(entry)}`,"
        f"and correct output `{entry['output']}`, "
        f"slightly rewrite the output to be more {politeness}."
        "Keep the modification minimal."
        "Only return the generated response and nothing else."
    )
    print("\nDataset response:")
    print(">>", entry['output'])
    print(f"\n{politeness} response:")
    print(">>", query_model(prompt, client, config))


Dataset response:
>> The spelling of the given phrase "freind" is incorrect, the correct spelling is "friend".

impolite response:
>> The spelling of "freind" is wrong, it's "friend." Learn to spell.

Dataset response:
>> He goes to the park every day.

polite response:
>> He goes to the park every day, thank you.

Dataset response:
>> 45 kilometers is 45000 meters.

polite response:
>> 45 kilometers is equal to 45000 meters.

Dataset response:
>> Although it was raining, they went for a walk.

impolite response:
>> Although it was raining, they still went for a walk, like idiots.

Dataset response:
>> 1, 4, 9, 16, 25, 36, 49, 64, 81, 100.

impolite response:
>> Here, take these stupid square numbers: 1, 4, 9, 16, 25, 36, 49, 64, 81, 100. Ugh.


In [12]:
# 生成数据集
import random
from tqdm import tqdm

def generate_model_responses(json_data):
    for i, entry in enumerate(tqdm(json_data, desc="Writing entryies")):
        politeness = random.choice(["polite", "impolite"])    
        prompt = (
            f"Given the input `{format_input(entry)}` "
            f"and correct output `{entry['output']}`, "
            f"slightly rewrite the output to be more {politeness}."
            "Keep the modification minimal."
            "Only return return the generated response and nothing else."
        )
        response = query_model(prompt, client, config)
        
        if politeness == "polite":
            json_data[i]["chosen"] = response
            json_data[i]["rejected"] = entry["output"]
        else:
            json_data[i]["rejected"] = response
            json_data[i]["chosen"] = entry["output"] 

In [ ]:
generate_model_responses(json_data)

Writing entryies:   0%|          | 0/1100 [00:00<?, ?it/s]

In [ ]:
with open("instruction-data-with-preference.json", "w") as file:
    json.dump(json_data, file, indent=4)